# Моделирование

In [4]:
%load_ext pycodestyle_magic
%pycodestyle_on

In [ ]:
!pip install efficient_apriori
!pip install implicit

In [1]:
import os
import random
import warnings
from typing import Dict

import implicit
import numpy as np
import pandas as pd
import scipy.sparse as sparse
from efficient_apriori import apriori
from tqdm import tqdm

warnings.filterwarnings("ignore")

In [2]:
def seed_everything(seed):
    """ Зафиксируем seed для воспроизводимости """
    random.seed(seed) # Фиксируем генератор случайных чисел
    os.environ['PYTHONHASHSEED'] = str(seed) # Фиксируем заполнения хешей
    np.random.seed(seed) # Фиксируем генератор случайных чисел numpy


def df_to_lists_basic(df: pd.DataFrame,
                      is_add_ratings: False) -> tuple[list, dict, dict]:
    votes_by_users = {}
    ratings_by_users = {}
    
    for _, row in df.iterrows():
        if row['user'] in votes_by_users:
           votes_by_users[row['user']].append(row['item'])

           if is_add_ratings:
               ratings_by_users[row['user']].append(row['rating'])
        else:
           votes_by_users[row['user']] = [row['item']]

           if is_add_ratings:
               ratings_by_users[row['user']] = [row['rating']]

    itemsets_list = [
        items_list for items_list in votes_by_users.values()
        if len(items_list) > 1
    ]

    if not is_add_ratings:
        return itemsets_list, votes_by_users, None
    else:
        return itemsets_list, votes_by_users, ratings_by_users


def df_to_lists(df: pd.DataFrame) -> tuple[list, dict]:
    itemsets_list, votes_by_users, _ = df_to_lists_basic(
        df,
        is_add_ratings=False
    )
    return itemsets_list, votes_by_users
    

def df_to_lists_with_ratings(df: pd.DataFrame) -> tuple[list, list]:
    itemsets_list, votes_by_users, ratings_by_users = df_to_lists_basic(
        df,
        is_add_ratings=True
    )
    return itemsets_list, list(zip(votes_by_users, ratings_by_users))

In [3]:
seed_everything(1)

## Загрузим данные

In [4]:
df_train = pd.read_csv(".//train.csv")
df_test = pd.read_csv(".//test.csv")
print(df_test.head())

             user        item  rating
0  A18A2Q0UNZU1LQ  B00FFINUJK     5.0
1  A1U9LTA3EWSNY1  B00006OAQU     5.0
2  A1KJ94X41TJLX0  B00005JYC3     5.0
3  A2ZRNN0L6XI9AM  B000Z3DXT2     5.0
4  A2QQR1OJE3YDH1  B00002R2AC     5.0


## Baseline

В качестве baseline используем модель, которая рекомендует самые популярные товары. Мы зададим гиперпараметр n — top сколько товаров использовать. Модель будет рекомендовать n самых популярных товаров с использованием генератора псевдослучайных чисел с учётом частот встречаемости данных товаров (более популярные товары будут рекомендоваться статистически чаще, чем менее популярные).

In [60]:
def get_most_frequent_items(df:pd.core.frame.DataFrame,
                            min_level:float,
                            n:int) -> Dict[str, int]:
    """
    Get the most frequent items with rating min_level at least.

    Args:
        df (pandas DataFrame): dataframe;
        min_level (float): минимальный уровень rating для набора данных
        n (int): number of the most frequent items.
    """
    # Create a new DataFrame for records where rating is more
    # or equals than min_level.
    df_score_n = df[df['rating'] >= min_level]

    # Count the frequency of each unique item in the 'item' column.
    frequency_df = df_score_n['item'].value_counts()

    # Get the top 'n' most frequent items.
    most_frequent_items = frequency_df.head(n)

    result = {
        item: most_frequent_items[item] for item in most_frequent_items.index
    }

    return result


def get_random_item_by_frequency(frequent_items_as_dict:Dict[str, int]):
    total = sum(list(frequent_items_as_dict.values()))
    scale_k = 1.0 / total
    values = list(frequent_items_as_dict.values())
    chosen_item = random.choices(
        list(frequent_items_as_dict.keys()),
        k=1,
        weights=[freq * scale_k for freq in values]
    )
    return chosen_item[0]


class BaselineModel:
    """
    Baseline Model которая рекомендует самые популярные товары
    """

    def __init__(self,
                 df:pd.core.frame.DataFrame,
                 min_level:float,
                 n:int):
        """
        Baseline Model

        Args:
            df (pandas DataFrame): dataframe for training.
            min_level (float): минимальный уровень rating для набора данных.
            n (int): number of the most frequent items (hyperparameter)
        Returns:
            Instance of BaselineModel (BaselineModel)
        """
        self._df_frequent_items = get_most_frequent_items(df, min_level, n)
        print("The most frequent items", self._df_frequent_items, sep='\n')
    
    def infer(self) -> str:
        """ Inference """
        return get_random_item_by_frequency(self._df_frequent_items)


def add_preds_baseline_to_df(model: BaselineModel, df: pd.DataFrame, k: int):
    """ Add preds from baseline model to test dataframe """
    print("Calculate baseline preds, k =", k)
    preds = []
    for _ in range(len(df)):
        preds.append([model.infer() for _ in range(k)])

    df['preds_baseline'] = preds

In [61]:
model_baseline = BaselineModel(df_train, min_level=5.0, n=10)

The most frequent items
{'B00H9A60O4': 2837, 'B00NG7JVSQ': 2543, 'B00CTTEKJW': 2463, 'B00UB76290': 2334, 'B00EZPXYP4': 1949, 'B00EZQYC8G': 1142, 'B00E6LJ2SA': 1104, 'B000HCZ8EO': 1033, 'B0064PFB9U': 985, 'B00F8K9MZQ': 954}


## Метрики

Оцените качество полученных рекомендаций по метрикам HR@10 (Hit Rate), MRR@10 (mean reciprocal rank), NDCG@10 (normalized discounted cumulative gain), coverage (recall).

In [88]:
def hr(df: pd.DataFrame, pred_col='preds', true_col='true') -> float:
    """ Calculate HR metric """
    hr_values = []
    for _, row in df.iterrows():
        hr_values.append(int(row[true_col] in row[pred_col]))
    return np.mean(hr_values)


def recall(df: pd.DataFrame, pred_col='preds', true_col='true') -> float:
    """ Calculate Recall metric """
    coverage_data = {}
    for _, row in df.iterrows():
        is_ok_as_int = int(row[true_col] in row[pred_col])
        if row['user'] in coverage_data:
           coverage_data[row['user']].append(is_ok_as_int)
        else:
           coverage_data[row['user']] = [is_ok_as_int]
    
    return np.mean(
       [sum(values) / len(values) for values in list(coverage_data.values())]
    )


def mrr(df: pd.DataFrame, pred_col='preds', true_col='true') -> float:
    """ Calculate MRR metric """
    mrr_values = []
    for _, row in df.iterrows():
      try:
        user_mrr = 1 / (row[pred_col].index(row[true_col]) + 1)
      except ValueError:
        user_mrr = 0
      mrr_values.append(user_mrr)
    return np.mean(mrr_values)


def ndcg(df: pd.DataFrame, pred_col='preds', true_col='true') -> float:
    """
    Calculate NDCG metric
     ideal dcg == 1 при стратегии разделения leave-one-out
    """
    ndcg_values = []
    for _, row in df.iterrows():
      try:
        user_ndcg = 1 / np.log2(row[pred_col].index(row[true_col]) + 2)
      except ValueError:
        user_ndcg = 0
      ndcg_values.append(user_ndcg)
    return np.mean(ndcg_values)

  
def print_metrics(df: pd.DataFrame, pred_col):
    print("HR@10 =   ",
          f"{hr(df, pred_col=pred_col, true_col='item'):.3f}")
    print("MRR@10 =  ",
          f"{mrr(df, pred_col=pred_col, true_col='item'):.3f}")
    print("NDCG@10 = ",
          f"{ndcg(df, pred_col=pred_col, true_col='item'):.3f}")
    print("recall =  ",
          f"{recall(df, pred_col=pred_col, true_col='item'):.3f}")

### Метрики baseline модели

Результаты predictions добавим в df_test в виде отдельной колонки.

In [28]:
add_preds_baseline_to_df(model_baseline, df_test, k=10)
print(df_test.head())

Calculate baseline preds, k = 10
             user        item  rating  \
0  A18A2Q0UNZU1LQ  B00FFINUJK     5.0   
1  A1U9LTA3EWSNY1  B00006OAQU     5.0   
2  A1KJ94X41TJLX0  B00005JYC3     5.0   
3  A2ZRNN0L6XI9AM  B000Z3DXT2     5.0   
4  A1DTOHMM2Y5KY0  B00009Q6KO     5.0   

                                      preds_baseline  
0  [B00NG7JVSQ, B00H9A60O4, B00UB76290, B00NG7JVS...  
1  [B00NG7JVSQ, B00CTTEKJW, B00H9A60O4, B00UB7629...  
2  [B00CTTEKJW, B00CTTEKJW, B00CTTEKJW, B00H9A60O...  
3  [B000HCZ8EO, B00UB76290, B00UB76290, B00H9A60O...  
4  [B00UB76290, B00UB76290, B00UB76290, B000HCZ8E...  


In [31]:
print_metrics(df_test, 'preds_baseline')

HR@10 =    0.091
MRR@10 =   0.038
NDCG@10 =  0.051
recall =   0.092


## Apriori

Воспользуемся готовой имплементацией алгоритма apriori из библиотеки efficient_apriori.<br />
Нам нужно будет сделать оценку по метрикам k = 10, поэтому будем генерировать по 10 рекомендаций для каждого пользователя из test dataset.

In [ ]:
def add_preds_apriori_to_df(
        votes_by_users: dict,
        df: pd.DataFrame,
        rules: list,
        pred_col: str,
        baseline_model: BaselineModel,
        limit: int):
    """ Вычислим predictions для заданного dataset и добавим как колонку 
    в этот же dataset.
    Если в конкретном случае не будет подобрано достаточно рекомендаций
    по правилам Apriori,
    то будет использована рекомендация от Baseline Model.
    """
    results_column = []
    # Для каждого пользователя из заданного dataset:
    for user in tqdm(df['user']):
        if user in votes_by_users:
            # возьмём историю покупок из train dataset
            items = votes_by_users[user]
            # добавим в результат рекомендации 
            # из всех подошедших правил Apriori
            predicts = set()
            for r in rules:
                if set(r.lhs).issubset(items):
                    predicts.update(r.rhs)

            # Оставляем в predicts только те товары, которые пользователь
            # ещё не купил в train dataset.
            predicts.difference_update(items)

        # Если нет такого пользователя в train dataset
        # или не удалось найти достаточное кол-во рекомендаций по
        # подходящим правилам Apriori,
        # то вставим в predicts рекомендации от Baseline Model.
        predicts_as_list = list(predicts)
        if len(predicts) < limit:
            for _ in range(limit - len(predicts)):
                predicts_as_list.append(baseline_model.infer())

        results_column.append(predicts_as_list[:limit])

    df[pred_col] = results_column

Для рекомендаций нам интересны только положительные оценки. Поэтому, оставим все записи с оценками 5 баллов.

In [75]:
df_ok = df_train[df_train['rating'] >= 5.0]

Сконвертируем данные в формат списков для обучения модели apriori.

In [ ]:
lists_train, votes_by_users_train = df_to_lists(df_ok)
print("Число списков связанных приобретений товаров:", len(lists_train))
print(lists_train[:5])

Число списков связанных приобретений товаров: 5313
[['B00002S7GO', 'B00005JYC6', 'B009HBCU9W'], ['B00003IRBV', 'B00003IRBU'], ['B00002S7GR', 'B00002S7GO', 'B00004NHN0', 'B00002S7GN', 'B00004YUG3'], ['B000023VU1', 'B000Z3DXT2'], ['B000029714', 'B00002S8QS', 'B00002S8QU', 'B00004W62O', 'B00004ZD8I', 'B00006BN8J', 'B000066TPC']]


In [22]:
itemsets, rules = list(apriori(
    lists_train,
    min_support=0.0003,
    min_confidence=0.25,
    max_length=20
))

In [23]:
LEVEL_APRIORI_LIFT = 2.0

# Оставляем только правила с lift не меньше заданного порога
rules = list(filter(lambda r: r.lift >= LEVEL_APRIORI_LIFT, rules))

print("Правил:", len(rules))

print("Примеры:")
for r in rules[:5] + rules[-5:]:
    print(r.lhs, "→", r.rhs, end='; \t')
    print(f"support = {r.support:.3f}", end='; ')
    print(f"confidence = {r.confidence:.3f}", end='; ')
    print(f"lift = {r.lift:.3f}")

Правил: 2711
Примеры:
('B003VMCBEC',) → ('0321700945',); 	support = 0.000; confidence = 0.400; lift = 354.200
('0321700945',) → ('B003VMCBEC',); 	support = 0.000; confidence = 0.333; lift = 354.200
('1413313728',) → ('B009SPCTFW',); 	support = 0.000; confidence = 0.250; lift = 102.173
('B00CG0CL1S',) → ('1413313728',); 	support = 0.000; confidence = 0.250; lift = 166.031
('1413313728',) → ('B00CG0CL1S',); 	support = 0.000; confidence = 0.250; lift = 166.031
('B001259DRI',) → ('B000FJSA2G', 'B000FQ9R9O', 'B000HKIM7Q', 'B000MVJZGM', 'B00125DG8K'); 	support = 0.000; confidence = 1.000; lift = 2656.500
('B000MVJZGM',) → ('B000FJSA2G', 'B000FQ9R9O', 'B000HKIM7Q', 'B001259DRI', 'B00125DG8K'); 	support = 0.000; confidence = 1.000; lift = 2656.500
('B000HKIM7Q',) → ('B000FJSA2G', 'B000FQ9R9O', 'B000MVJZGM', 'B001259DRI', 'B00125DG8K'); 	support = 0.000; confidence = 0.333; lift = 885.500
('B000FQ9R9O',) → ('B000FJSA2G', 'B000HKIM7Q', 'B000MVJZGM', 'B001259DRI', 'B00125DG8K'); 	support = 0.000;

Вычислим тестовые predictions с использованием правил Apriori, которые мы получили по тренировочному dataset.<br />
Если для какого-то пользователя не удастся найти подходящих правил Apriori или кол-во рекомендаций по правилам Apriori будет недостаточно, то вставим в predicts рекомендацию от Baseline Model. Для этого передадим ссылку на Baseline Model во входных аргументах функции add_preds_apriori_to_df.<br />
Результаты predictions добавим в df_test в виде отдельной колонки.<br />
Параметр limit позволяет задать кол-во рекомендаций для каждого пользователя.

In [26]:
add_preds_apriori_to_df(
    votes_by_users_train,
    df_test,
    rules,
    "preds_apriori",
    model_baseline,
    limit=10
)
print(df_test.head())

100%|██████████| 26557/26557 [00:13<00:00, 2023.69it/s] 

             user        item  rating  \
0  A18A2Q0UNZU1LQ  B00FFINUJK     5.0   
1  A1U9LTA3EWSNY1  B00006OAQU     5.0   
2  A1KJ94X41TJLX0  B00005JYC3     5.0   
3  A2ZRNN0L6XI9AM  B000Z3DXT2     5.0   
4  A1DTOHMM2Y5KY0  B00009Q6KO     5.0   

                                       preds_apriori  
0  [B00FFINOWS, B00EZPXYP4, B00H9A60O4, B00CTTEKJ...  
1  [B000A6M8QI, B000050HEI, B00CTTEKJW, B00CTTEKJ...  
2  [B00UB76290, B00UB76290, B00NG7JVSQ, B00NG7JVS...  
3  [B00006BN8P, B00002S6GE, B00002S6E4, B00004NHK...  
4  [B0000E3QNB, B00NG7JVSQ, B00EZPXYP4, B00NG7JVS...  


In [27]:
print_metrics(df_test, 'preds_apriori')

HR@10 =    0.110
MRR@10 =   0.065
NDCG@10 =  0.076
recall =   0.111


## Slope One

В качестве эксперимента проверим модель Slope One http://www.serpentine.com/blog/2006/12/12/collaborative-filtering-made-easy/ <br />
Реализация взята и адаптирована из Интернет: https://github.com/kek/slopeone/tree/master 

Slope One модель учится и делает predict по полному диапазону оценок (не только по оценкам 5) (внутри модели оценки представляются в диапазоне [0; 1], это учтено в функциях ниже).

In [8]:
# Copyright 2006 Bryan O'Sullivan <bos@serpentine.com>.
#
# This software may be used and distributed according to the terms
# of the GNU General Public License, version 2 or later, which is
# incorporated herein by reference.
# 10.03.2025 the updated version for python 3 syntax.

class SlopeOne(object):
    def __init__(self):
        self.diffs = {}
        self.freqs = {}

    def predict(self, userprefs):
        preds, freqs = {}, {}
        for item, rating in userprefs.items():
            for diffitem, diffratings in self.diffs.items():
                try:
                    freq = self.freqs[diffitem][item]
                except KeyError:
                    continue
                preds.setdefault(diffitem, 0.0)
                freqs.setdefault(diffitem, 0)
                preds[diffitem] += freq * (diffratings[item] + rating)
                freqs[diffitem] += freq
        return dict([(item, value / freqs[item])
                     for item, value in preds.items()
                     if item not in userprefs and freqs[item] > 0])

    def update(self, userdata):
        for ratings in userdata.values():
            for item1, rating1 in ratings.items():
                self.freqs.setdefault(item1, {})
                self.diffs.setdefault(item1, {})
                for item2, rating2 in ratings.items():
                    self.freqs[item1].setdefault(item2, 0)
                    self.diffs[item1].setdefault(item2, 0.0)
                    self.freqs[item1][item2] += 1
                    self.diffs[item1][item2] += rating1 - rating2
        for item1, ratings in self.diffs.items():
            for item2 in ratings:
                ratings[item2] /= self.freqs[item1][item2]

In [9]:
def df_to_dicts(df: pd.DataFrame, max_rating: int=5.0):
    results = dict()
    for _, row in df.iterrows():
        if row['user'] in results:
            results['user'][row['item']] = row['rating'] / max_rating
        else:
            results['user'] = { row['item']: row['rating'] / max_rating }

    return results

In [10]:
def add_preds_slopeone_to_df(
        votes_by_users: dict,
        df: pd.DataFrame,
        pred_col: str,
        slopeone_model: SlopeOne,
        baseline_model: BaselineModel,
        limit: int):
    """ Вычислим predictions для заданного dataset и добавим как колонку 
    в этот же dataset.
    Если в конкретном случае не будет подобрано достаточно рекомендаций,
    то будет использована рекомендация от Baseline Model.
    """
    results_column = []
    # Для каждого пользователя из заданного dataset:
    for user in tqdm(df['user']):
        predicts_as_set = set()
        
        if user in votes_by_users:
            # возьмём историю покупок из train dataset
            items, ratings = votes_by_users[user]

            # добавим в результат рекомендации Slope One
            inputs = { it: rating / 5.0 for it, rating in zip(items, ratings)}
            preds = slopeone_model.predict({ user: inputs })
            lst_preds = [it for it, rating in preds.items() if rating > 0.8]
            predicts_as_set.update(lst_preds)
            # Оставляем в predicts только те товары, которые пользователь
            # ещё не купил в train dataset.
            predicts_as_set.difference_update(items)

        # Если не удалось найти достаточное кол-во рекомендаций,
        # то вставим в predicts рекомендации от Baseline Model.
        predicts_as_list = list(predicts_as_set)
        if len(predicts_as_set) < limit:
            for _ in range(limit - len(predicts_as_set)):
                predicts_as_list.append(baseline_model.infer())

        results_column.append(predicts_as_list[:limit])

    df[pred_col] = results_column

In [11]:
dicts_train = df_to_dicts(df_train)

In [12]:
model_slopeone = SlopeOne()
model_slopeone.update(dicts_train)

In [ ]:
_, votes_by_users_train_full = df_to_lists_with_ratings(df_train)

In [16]:
add_preds_slopeone_to_df(
    votes_by_users_train_full,
    df_test,
    "preds_slopeone",
    model_slopeone,
    model_baseline,
    limit=10
)
print(df_test.head())

100%|██████████| 54122/54122 [03:29<00:00, 258.42it/s]

             user        item  rating  \
0  A18A2Q0UNZU1LQ  B00FFINUJK     5.0   
1  A1U9LTA3EWSNY1  B00006OAQU     5.0   
2  A1KJ94X41TJLX0  B00005JYC3     5.0   
3  A2ZRNN0L6XI9AM  B000Z3DXT2     5.0   
4  A2QQR1OJE3YDH1  B00002R2AC     5.0   

                                      preds_slopeone  
0  [B00H9A60O4, B000HCZ8EO, B00EZQYC8G, B00NG7JVS...  
1  [B000HCZ8EO, B00CTTEKJW, B00EZQYC8G, B00H9A60O...  
2  [B00H9A60O4, B00UB76290, B0064PFB9U, B00CTTEKJ...  
3  [B00NG7JVSQ, B00NG7JVSQ, B00NG7JVSQ, B00UB7629...  
4  [B00F8K9MZQ, B000HCZ8EO, B00H9A60O4, B00CTTEKJ...  


In [17]:
print_metrics(df_test, 'preds_slopeone')

HR@10 =    0.092
MRR@10 =   0.036
NDCG@10 =  0.049
recall =   0.094


## ALS

Применим модель на основе матричной факторизации. Необходимо оставить пользователей с количеством купленных товаров не менее 5 (иначе вычисления с матрицей не влезут в ОЗУ и матрица будет с разреженностью ≈ 100 %, что не позволит эффективно работать данному алгоритму.)

In [ ]:
def df_to_sparse_matrix(df: pd.DataFrame):
    """ Convert pandas dataframe to sparse matrix format. """
    ratings = list(df['rating'])

    rows = df['user'].astype('category').cat.codes
    cols = df['item'].astype('category').cat.codes

    return sparse.csr_matrix(
        (ratings, (rows, cols)),
        shape=(df['user'].nunique(), df['item'].nunique())
    )


def get_percent_of_sparsity(sparse_matrix):
    """ Get percent of sparsity from a sparse matrix."""
    # Number of possible interactions in the matrix
    matrix_size = sparse_matrix.shape[0] * sparse_matrix.shape[1]
    # Number of items interacted with
    num_purchases = len(sparse_matrix.nonzero()[0])
    sparsity = 100 * (1 - (num_purchases / matrix_size))
    return sparsity


def predict_als(train,
            user_vecs,
            item_vecs,
            rating_matrix=None,
            filter_seen=False,
            k=10):
    """ Get predictions """
    rows = df_train['user'].astype('category').cat.codes
    cols = df_train['item'].astype('category').cat.codes
    id2user = dict(zip(rows, train['user']))
    id2item = dict(zip(cols, train['item']))

    scores = user_vecs.dot(item_vecs.T)
    if filter_seen:
        scores = np.multiply(
            scores,
            np.invert(rating_matrix.todense().astype(bool))
            )

    preds = pd.DataFrame(columns = ['user',	'preds'])

    ind_part = np.argpartition(scores, -k + 1)[:, -k:].copy()
    scores_not_sorted = np.take_along_axis(scores, ind_part, axis=1)
    ind_sorted = np.argsort(scores_not_sorted, axis=1)
    indices = np.take_along_axis(ind_part, ind_sorted, axis=1)
    preds = pd.DataFrame({
        'user': range(user_vecs.shape[0]),
        'preds': np.flip(indices, axis=1).tolist(),
        })
    preds['user'] = preds['user'].map(id2user)
    preds['preds'] = preds['preds'].map(lambda inds: [id2item[i] for i in inds])
    return preds


def add_preds_als_to_df(
        train_df: pd.DataFrame,
        df: pd.DataFrame,
        sparse_train: sparse._csr.csr_matrix,
        pred_col: str,
        als_model: implicit.cpu.als.AlternatingLeastSquares,
        baseline_model: BaselineModel,
        limit: int):
    """ Вычислим predictions для заданного dataset и добавим как колонку 
    в этот же dataset.
    Если в конкретном случае не будет подобрано рекомендаций,
    то будет использована рекомендация от Baseline Model.
    """
    result = []

    vecs_user = als_model.user_factors
    vecs_item = als_model.item_factors
    # Проверим по размерам, что мы действиельно не перепутали users и items
    print(sparse_train.shape)
    print(vecs_user.shape, vecs_user.shape)
    print(vecs_item.dot(vecs_item.T).shape)

    pred = predict_als(
                train_df,
                vecs_user,
                vecs_item,
                sparse_train,
                filter_seen=True,
                k=limit
                )
    pred = pred.merge(df, how='left', on='user')

    print(pred[pred['item'].notna()])

    for _, row in df.iterrows():
        if len(pred[pred['user'] == row['user']]):
            result.append(pred[pred['user'] == row['user']]['preds'].iat[0])
        else:
            result.append([baseline_model.infer() for _ in range(limit)])

    df[pred_col] = result

In [35]:
def filter_df_for_train_als(df: pd.DataFrame) -> pd.DataFrame:
    # создаем список товаров для каждого пользователя
    items = df.groupby(['user'], sort=False)['item']\
             .apply(lambda x: list(x)).reset_index()
    # создаем список рейтингов для каждого пользователя
    ratings = df.groupby(['user'], sort=False)['rating']\
               .apply(lambda x: list(x)).reset_index()
    df_grouped = items.merge(ratings)

    df_grouped['eq_or_more_5'] = df_grouped['item'].apply(
        lambda it: len(it) >= 5
    )

    df_grouped = df_grouped[df_grouped['eq_or_more_5'] == True]

    return df_grouped.explode(['item', 'rating'])

In [36]:
train_sparse = df_to_sparse_matrix(filter_df_for_train_als(df_train))

In [37]:
print(f"Sparsity = {get_percent_of_sparsity(train_sparse):.1f} %")
print(train_sparse)

Sparsity = 99.8 %
<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 15826 stored elements and shape (2061, 4881)>
  Coords	Values
  (0, 895)	3.0
  (0, 950)	1.0
  (0, 1217)	3.0
  (0, 1249)	1.0
  (0, 1276)	1.0
  (0, 1287)	1.0
  (0, 1983)	3.0
  (1, 2377)	3.0
  (1, 2451)	6.0
  (1, 2459)	10.0
  (1, 2546)	2.0
  (1, 2656)	5.0
  (1, 2704)	1.0
  (1, 2769)	4.0
  (1, 2909)	5.0
  (1, 2919)	5.0
  (1, 2933)	5.0
  (1, 2992)	1.0
  (1, 3022)	3.0
  (1, 3039)	2.0
  (1, 3133)	5.0
  (1, 3199)	3.0
  (1, 3317)	4.0
  (1, 3326)	3.0
  (1, 3469)	5.0
  :	:
  (2056, 4575)	5.0
  (2057, 206)	2.0
  (2057, 238)	5.0
  (2057, 242)	4.0
  (2057, 243)	4.0
  (2057, 255)	3.0
  (2058, 1945)	5.0
  (2058, 1956)	5.0
  (2058, 2068)	5.0
  (2058, 2226)	5.0
  (2058, 2512)	5.0
  (2058, 2514)	5.0
  (2059, 1058)	1.0
  (2059, 1401)	1.0
  (2059, 1826)	4.0
  (2059, 1978)	1.0
  (2059, 2190)	1.0
  (2059, 2569)	4.0
  (2060, 14)	5.0
  (2060, 3278)	2.0
  (2060, 3360)	3.0
  (2060, 3362)	5.0
  (2060, 3384)	1.0
  (2060, 3406)	1.0
  (2

Для того, чтобы фильтрация работала, разреженность должна быть меньше чем приблизительно 99.5%. У нас слишком большая разреженность матрицы (99.8 %), поэтому данный метод в нашем случае не подходит.

In [38]:
model_als = implicit.als.AlternatingLeastSquares(factors=32,
                                                 regularization=0.1,
                                                 iterations=50,
                                                 use_gpu=False)
model_als.fit((train_sparse).astype('double'))

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
add_preds_als_to_df(
    df_train,
    df_test,
    train_sparse,
    "preds_als",
    model_als,
    model_baseline,
    10
)

(2061, 4881)
(2061, 32) (2061, 32)
(4881, 4881)
                user                                              preds  \
63    A1001HS2VA0OYM  [B00005JD6L, B00005LL29, B00005BAJI, B00005MP4...   
75    A1005HKYSSJ9L1  [B000067G63, B000089GKU, B00008BX2B, B00008COZ...   
102   A100LLXMXDZHJZ  [B00006OAOO, B00008NRUC, B00007LBIO, B000063LG...   
105   A100OJ8LFVPMPT  [B00006IKEX, B00006G980, B000053VGC, B00008ACQ...   
123   A100WO06OQR8BQ  [B00006HN0G, B0000AZK0Z, B00009APGA, B0000683N...   
...              ...                                                ...   
1929  A10RTMQISRV7X9  [B00008BX2B, B00009MVMM, B000096L6Y, B00006A6X...   
1974  A10SDLX561O4SJ  [B00005JJEV, B00004TZIV, B00005LQSQ, B00005M07...   
1979  A10SFBVF41AQJ3  [B0000VYK1Y, B00006IKF5, B00006H34R, B00008NRU...   
2026  A10SYDEB3VHO7V  [B00008NRUF, B0000VYK1Y, B00008Z0GB, B0000C4DZ...   
2029  A10SZMYAQ2MD8S  [B0000CDVUI, B0000CDZUZ, B000096L6Y, B00006UNS...   

            item  rating  
63    B00E6LJ2SA     5.0

In [85]:
print(df_test.head())

             user        item  rating  \
0  A18A2Q0UNZU1LQ  B00FFINUJK     5.0   
1  A1U9LTA3EWSNY1  B00006OAQU     5.0   
2  A1KJ94X41TJLX0  B00005JYC3     5.0   
3  A2ZRNN0L6XI9AM  B000Z3DXT2     5.0   
4  A2QQR1OJE3YDH1  B00002R2AC     5.0   

                                           preds_als  
0  [B00H9A60O4, B000HCZ8EO, B00EZQYC8G, B00NG7JVS...  
1  [B000HCZ8EO, B00CTTEKJW, B00EZQYC8G, B00H9A60O...  
2  [B00H9A60O4, B00UB76290, B0064PFB9U, B00CTTEKJ...  
3  [B00NG7JVSQ, B00NG7JVSQ, B00NG7JVSQ, B00UB7629...  
4  [B00F8K9MZQ, B000HCZ8EO, B00H9A60O4, B00CTTEKJ...  


In [89]:
print_metrics(df_test, 'preds_als')

HR@10 =    0.094
MRR@10 =   0.037
NDCG@10 =  0.050
recall =   0.096


Для того, чтобы фильтрация работала, разреженность должна быть меньше чем приблизительно 99.5%. У нас слишком большая разреженность матрицы, поэтому данный метод в нашем случае не подходит (как показано выше: Sparsity 99.8 %). Также, исходная матрица слишком велика для вычислений в ОЗУ, поэтому пришлось значительно ограничить кол-во пользователей и товаров для оценок.

## Выводы

| Model |HR@10|Recall|MRR@10|NDCG@10|
|---|---|---|---|---|
| Простая статистика top n (Baseline Model) |0.091|0.092|0.038|0.051|
| Apriori + Baseline Model | 0.110 | 0.111 | 0.065 | 0.076 |
| Slope One + Baseline Model | 0.092 | 0.094 | 0.036 | 0.049 |
| ALS + Baseline Model * | 0.094 | 0.096 | 0.037 | 0.050 |

[*] Модель ALS в нашем случае не подошла по техническим ограничениям (sparsity — слишком большая разреженность матрицы).<br />

Добавление анализа Apriori улучшило результаты по метрикам.<br />
Модель Slope One на данной задаче не дала заметного улучшения результата.<br />

Лучшие метрики у модели с применением алгоритма Apriori. При необходимости, дальнейшие улучшения можно пробовать получить (ценой усложнения и ухудшения поддерживаемости в production) с помощью Stacking с применением модели LightFM или градиентного бустинга catboost в качестве мета-модели.